# param recovery dual Q

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import confusion_matrix
# np.random.seed(42)
from joblib import Parallel, delayed
import matplotlib.tri as tri
import matplotlib.colors as mcolors
from scipy.interpolate import griddata
from scipy.interpolate import RBFInterpolator
import matplotlib.ticker as mticker
import itertools
from sklearn.metrics import r2_score


In [2]:
output_dir = "27_RL_agent_TDlearn_output_both_param_recovery"
os.makedirs(output_dir, exist_ok=True)


folder_path_participants = 'data_risk_added_epileptic'
folder_path_colors_numbers = '13_RL_agent_TDlearn_output_risk_dualQ/model_behavior'


df_participants = []
df_colors_numbers = []


def find_matching_csv(folder_path, df_list):
            for csv_file in os.listdir(folder_path):
                if clean_name in csv_file and csv_file.endswith('.csv'):
                    csv_path = os.path.join(folder_path, csv_file)
                    df_csv = pd.read_csv(csv_path)
                    df_list.append(df_csv)





for file_name in os.listdir(folder_path_participants):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path_participants, file_name)
        df = pd.read_csv(file_path)
        df = df[df['outcome'].str.lower() != 'na'].reset_index(drop=True) 
        df_participants.append(df)

        clean_name = file_name.removeprefix("task_data_").removesuffix(".csv")
        find_matching_csv(folder_path_colors_numbers, df_colors_numbers)


In [3]:
for df in df_participants:
    df['block_type'] = None

    df.loc[df['block'] == 1, 'block_type'] = 'uniform'     # Block 1 is uni
    df.loc[df['block'] == 4, 'block_type'] = 'mix'     # Block 4 is mix

    # For blocks 2 and 3, set based on distribution
    df.loc[(df['block'] == 2) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 2) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'low'), 'block_type'] = 'low'
    df.loc[(df['block'] == 3) & (df['distribution'] == 'high'), 'block_type'] = 'high'
    



for i in range(len(df_participants)):
    myCard = df_participants[i]['myCard']
    yourCard = df_participants[i]['yourCard']
    distributions = df_participants[i]['distribution']
    block_type = df_participants[i]['block_type']
    risk = df_participants[i]['risk']
    
    for df_list in [ df_colors_numbers]:
        df_list[i]['myCard'] = myCard
        df_list[i]['yourCard'] = yourCard
        df_list[i]['distribution'] = distributions
        df_list[i]['block_type'] = block_type
        df_list[i]['risk'] = risk

In [4]:
for df in df_colors_numbers:
    df['model_choices'] = df['model_choices'].replace({1: 'arrowup', 0: 'arrowdown'})

In [5]:
for df in df_colors_numbers:
    outcomes = []
    for i in range(len(df)):
        my = df.loc[i, 'myCard']
        your = df.loc[i, 'yourCard']
        choice = df.loc[i, 'model_choices']
        
        if ((my > your and choice == "arrowup") or (my < your and choice == "arrowdown")):
            outcomes.append('win')
        else:
            outcomes.append('lose')
    
    df['outcome'] = outcomes

In [6]:
participants = [os.path.splitext(file)[0].replace("task_data_", "")
    for file in os.listdir(folder_path_participants) if file.endswith('.csv')]

In [7]:
actions = { "arrowdown": 0, "arrowup": 1}
distributions_map = { "uniform": 0, "low": 1,  "high": 2}
card_numbers = list(range(1, 10))

# policy_table = percentage_matrix 

Q_table_init = np.random.normal(0, 0.1, (len(card_numbers), len(distributions_map), len(actions)))
# having a q-table based on the policies
# Q_table_init = policy_table * np.mean(Q_table_init) 

Q_table = Q_table_init.copy()

#############################################################################################
# having a q-table that starts with 0! this was not a good initilization so i changed it.
# Q_table = np.zeros((len(distributions_map), len(actions)))  # 3 distributions × 2 actions
#############################################################################################

# print("policy: \n",np.shape(policy_table))
print("\n Q_table: \n",np.shape(Q_table))




 Q_table: 
 (9, 3, 2)


In [8]:
df_colors_numbers[0]

,model_choices,participant_choices,model_total_reward,participant_total_reward,q_val,myCard,yourCard,distribution,block_type,risk,outcome
0,arrowdown,0,10.5,10.5,"[[[0.09769494872300748, -0.027194159319305494]...",2,7,uniform,uniform,0.125,win
1,arrowdown,1,10.0,11.0,"[[[0.09769494872300748, -0.027194159319305494]...",9,4,uniform,uniform,0.000,lose
2,arrowup,0,9.5,11.5,"[[[0.09769494872300748, -0.027194159319305494]...",5,6,uniform,uniform,0.500,lose
3,arrowup,1,10.0,12.0,"[[[0.09769494872300748, -0.027194159319305494]...",9,1,uniform,uniform,0.000,win
4,arrowup,1,10.5,12.5,"[[[0.09769494872300748, -0.027194159319305494]...",6,4,uniform,uniform,0.375,win
...,...,...,...,...,...,...,...,...,...,...,...
265,arrowup,1,61.0,85.0,"[[[0.4964520000302995, -0.027194159319305494],...",8,7,high,mix,0.243,win
266,arrowup,0,60.5,85.5,"[[[0.4964520000302995, -0.027194159319305494],...",4,8,high,mix,0.146,lose
267,arrowup,1,60.0,85.0,"[[[0.4964520000302995, -0.027194159319305494],...",8,9,low,mix,0.023,lose
268,arrowup,0,60.5,84.5,"[[[0.4964520000302995, -0.027194159319305494],...",3,1,low,mix,0.447,win


In [9]:
def softmax(Q_values, beta):
    # this part subtracts the maximum q-value in each row it means each state to improve numerical stability.
    # because exxponentials of large numbers can lead to overflow errors, so shifting q-values avoids this problem.
    
    Q_shifted = Q_values - np.max(Q_values, axis=2, keepdims=True)
    exps = np.exp(beta * Q_shifted)
    sums = np.sum(exps, axis=2, keepdims=True)
    new_probs = exps / sums

    return new_probs




def train_dualQ_risk_sensitive(df, alpha_r, alpha_s, beta, eta, Qr_init=None, Qs_init=None):
    if Qr_init is None:
        Qr_init = Q_table.copy()
    if Qs_init is None:
        Qs_init = Q_table.copy()

    Qr = Qr_init.copy()
    Qs = Qs_init.copy()

    q_value_pairs = []
    choices = []
    predicted_probs = []
    distributions = []
    card_numbers = []

    for _, row in df.iterrows():
        action = actions[row["model_choices"]]
        distribution = distributions_map[row["distribution"]]
        card_number = row["myCard"] -1
        reward = 0.5 if row["outcome"] == "win" else -0.5
        risk = row["risk"]  

        Q_combined = Qr - eta * Qs
        probs = softmax(Q_combined, beta)
        predicted_probs.append(probs[card_number][distribution][action])

        Qr[card_number][distribution][action] += alpha_r * (reward - Qr[card_number][distribution][action])
        Qs[card_number][distribution][action] += alpha_s * (risk - Qs[card_number][distribution][action])

        q_value_pairs.append(Q_combined.copy())
        choices.append(action)
        distributions.append(distribution)
        card_numbers.append(card_number)

    return np.array(q_value_pairs), np.array(choices), np.array(predicted_probs), np.array(distributions), np.array(card_numbers)



def compute_log_likelihood(alpha_r, alpha_s, beta, eta, df_all):
    q_values, choices, predicted_probs, distributions, card_numbers = train_dualQ_risk_sensitive(
        df_all, alpha_r, alpha_s, beta, eta
    )
    predicted_probs = np.clip(predicted_probs, 1e-6, 1)
    log_likelihood = np.sum(np.log(predicted_probs))
    return (alpha_r, alpha_s, beta, eta, log_likelihood)



In [10]:
num_of_samples = 50
alpha_min = 0.01
alpha_max = 1
beta_min = 0.01
beta_max  = 8
eta_min = -1
eta_max = 1

alpha_r_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
alpha_s_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
beta_samples = np.random.uniform(beta_min, beta_max + np.finfo(float).eps, num_of_samples)
eta_samples = np.random.uniform(eta_min, eta_max + np.finfo(float).eps, num_of_samples)

In [11]:
BIC_models = []
AIC_models = []
best_alpha_r_models = []
best_alpha_s_models = []
best_eta_models = []
best_beta_models = []
accuracy_models = []
precision_models = []
sensitivity_recall_models = []
specificity_models = []
f1_score_models = []
mcFadden_r2_models = []
r2_models = []

for idx, df_all in enumerate(df_colors_numbers):
    print(f"Processing participant {idx + 1} of {len(df_colors_numbers)}")
    Q_init_participant = Q_table.copy()
    
    df_all = df_all[df_all['outcome'].str.lower() != 'na'].reset_index(drop=True)


    results = Parallel(n_jobs=-1, backend='loky')(
    delayed(compute_log_likelihood)(alpha_r, alpha_s, beta, eta, df_all)
    for alpha_r in alpha_r_samples
    for alpha_s in alpha_s_samples
    for beta in beta_samples
    for eta in eta_samples)



    alpha_beta_log_likelihood = {}
    best_log_likelihood = -np.inf


    for alpha_r, alpha_s, beta, eta, log_likelihood in results:
        if log_likelihood > best_log_likelihood:
            best_log_likelihood = log_likelihood
            best_alpha_r = alpha_r
            best_alpha_s = alpha_s
            best_beta = beta
            best_eta = eta



    # model prediction 
    
    q_values, choices, predicted_probs, distributions, card_numbers = train_dualQ_risk_sensitive(df_all, best_alpha_r, best_alpha_s, best_beta, best_eta)

    
    
    predicted_choices = []
    for trial in range(len(card_numbers)):
        test_action_probs = softmax(q_values[trial], best_beta)
        p_arrowup = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowup"]]
        p_arrow_down = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowdown"]]
        # choosing 1 or 0 based on the softmax probabilities:
        predicted_choices.append(np.random.choice([1, 0], p=[p_arrowup, p_arrow_down]))


    # finding out model total reward based on the model's predicted choices
    total_reward = [] 
    for i in range(len(predicted_choices)):
        if len(total_reward)> 0:
            last_reward = total_reward[-1]  #  the last reward value
        else:
            last_reward = 10 # initial reward is $10
        
        if ((df_all.loc[i, 'myCard'] > df_all.loc[i, 'yourCard'] and predicted_choices[i] == 1) or
            (df_all.loc[i, 'myCard'] < df_all.loc[i, 'yourCard'] and predicted_choices[i] == 0)):
            total_reward.append(last_reward + 0.5)
        else:
            total_reward.append(last_reward - 0.5)

    
    
    # confusion matrix:
    conf_matrix = confusion_matrix(choices, predicted_choices)
    TN, FP, FN, TP = conf_matrix.ravel()  # unpacking the confusion matrix
    # acc
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    # precision: From the ones that we’ve announced them as up/down, which ones are really up/down?
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    # recall or sensitivity : true positive rate
    sensitivity_recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    # specificity : true negative rate
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    # f1 Score
    f1_score = 2 * (precision * sensitivity_recall) / (precision + sensitivity_recall) if (precision + sensitivity_recall) != 0 else 0

    
    # bayes information criterion:
    n_trials = len(df_all)
    k = 4  # number of free parameters: alpha and beta
    BIC = k * np.log(n_trials) - 2 * best_log_likelihood # this is BIC formula based on the log lkelihode I found before

        # Akaike  information criterion(AIC):
    AIC = 2 * k - 2 * best_log_likelihood 

    # mcFadden r-squared:
    p_null = np.mean(choices)  # probability of choosing "1" in the dataset
    log_likelihood_null = np.sum(choices * np.log(p_null) + (1 - choices) * np.log(1 - p_null))
    mcFadden_r2 = 1 - (best_log_likelihood / log_likelihood_null)

    # r-squared
    r2 = r2_score(choices, predicted_choices)
    
    
    # saving models evaluation variables:
    best_alpha_r_models.append(best_alpha_r)
    best_alpha_s_models.append(best_alpha_s)
    best_eta_models.append(best_eta)
    best_beta_models.append(best_beta)
    BIC_models.append(BIC)
    AIC_models.append(AIC)
    accuracy_models.append(accuracy)
    precision_models.append(precision)
    sensitivity_recall_models.append(sensitivity_recall)
    specificity_models.append(specificity)
    f1_score_models.append(f1_score)
    mcFadden_r2_models.append(mcFadden_r2)
    r2_models.append(r2)
    print(accuracy)

  

Processing participant 1 of 8
0.6333333333333333
Processing participant 2 of 8
0.6296296296296297
Processing participant 3 of 8
0.7666666666666667
Processing participant 4 of 8
0.5703703703703704
Processing participant 5 of 8
0.6518518518518519
Processing participant 6 of 8
0.6444444444444445
Processing participant 7 of 8
0.6259259259259259
Processing participant 8 of 8
0.5888888888888889


# now saving the model evaluation values

In [12]:
df_models_evaluation = pd.DataFrame({
    "participants": participants,
    "best_alpha_r": best_alpha_r_models,
    "best_alpha_s": best_alpha_s_models,
    "best_beta": best_beta_models,
    "best_eta": best_eta_models,
    "BIC": BIC_models,
    "AIC": AIC_models,
    "accuracy": accuracy_models,
    "precision": precision_models,
    "sensitivity_recall": sensitivity_recall_models,
    "specificity": specificity_models,
    "f1_score": f1_score_models,
    "mcFadden_r2": mcFadden_r2_models,
    "r2": r2_models
})

file_path = os.path.join(output_dir, "models_evaluation_risk_dualQ.csv")
df_models_evaluation.to_csv(file_path, index=False)